In [1]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow_datasets as tfds

In [3]:
imdb,info=tfds.load("imdb_reviews",with_info=True,as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete8MOAIY/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete8MOAIY/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete8MOAIY/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
train_set,test_set=imdb['train'],imdb['test']

In [5]:
training_sentences=[]
training_labels=[]
testing_labels=[]
testing_sentences=[]

for s,l in train_set:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())
for s,l in test_set:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())


In [6]:
import numpy as np
training_labels=np.array(training_labels)
testing_labels=np.array(testing_labels)

In [7]:
max_length=120
token=Tokenizer(num_words=10000,oov_token='00V')
token.fit_on_texts(training_sentences)
word_labels=token.word_index
sequence=token.texts_to_sequences(training_sentences)
padded=pad_sequences(sequence,maxlen=max_length,truncating='post')
test_sequence=token.texts_to_sequences(testing_sentences)
test_padded=pad_sequences(test_sequence,maxlen=max_length,truncating='post')


In [8]:

print(len(word_labels))

86539


In [9]:
print(padded.shape)

(25000, 120)


In [10]:
reverse_word_index = dict([(value, key) for (key, value) in word_labels.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? b'this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you 00V into a big arm chair and 00V for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no 00V to cross no dangerous waters just a warm and witty 00V through new york life at its best a family film in every sense and one that deserves the praise it received '
b'This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.'


In [11]:
from tensorflow.keras.layers import Embedding,Flatten,Dense,Bidirectional,LSTM,GRU
from tensorflow.keras.models import Sequential

In [12]:
model=Sequential()
model.add(Embedding(10000,16,input_length=120))
model.add(Bidirectional(LSTM(64,return_sequences=True)))
model.add(Bidirectional(LSTM(32,return_sequences=True)))
model.add(Bidirectional(LSTM(16,return_sequences=True)))
model.add(Bidirectional(LSTM(8)))
model.add(Flatten())
model.add(Dense(6,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [13]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
bidirectional (Bidirectional (None, 120, 128)          41472     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 120, 64)           41216     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 120, 32)           10368     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 16)                2624      
_________________________________________________________________
flatten (Flatten)            (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 6)                 1

In [ ]:
history=model.fit(padded,training_labels,epochs=50,validation_data=(test_padded,testing_labels))

Epoch 1/50
782/782 [==============================] - 48s 61ms/step - loss: 0.5344 - accuracy: 0.7339 - val_loss: 0.4359 - val_accuracy: 0.7991
Epoch 2/50
782/782 [==============================] - 46s 59ms/step - loss: 0.3501 - accuracy: 0.8548 - val_loss: 0.4336 - val_accuracy: 0.8078
Epoch 3/50
159/782 [=====>........................] - ETA: 24s - loss: 0.2899 - accuracy: 0.8911

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
np.linalg.norm()

In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc))
#plotting the accuracy

plt.plot(acc)
plt.plot(val_acc)
plt.title("accuracy measure")
plt.xlabel("epochs")
plt.legend(["accuracy","val_accuracy"])
plt.ylabel("accuracies")

plt.figure()

plt.plot(loss)
plt.plot(val_loss)
plt.title("loss measure")
plt.xlabel("epochs")
plt.legend(["loss","val_loss"])
plt.ylabel("losses")


In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

In [ ]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, 10000):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [ ]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')